[Dokumentacja mysql.connector](https://dev.mysql.com/doc/connector-python/en/connector-python-example-connecting.html)

Funkcja connect() tworzy połączenie z MySQL server i zwraca obiekt MySQLConnection:
```python
cnx = mysql.connector.connect()
```

Metoda połączenia cursor() zwraca obiekt MySQLCursor, który umożliwia manipulowanie bazą:
```python
cursor = cnx.cursor()
cursor.execute(sql_statement)
```

Po dodaniu (INSERT INTO), zaktualizowaniu (UPDATE) lub usunięciu (DELETE FROM) wierszy, zmiany należy jeszcze skomitować:
```python
cnx.commit()
```

Połączenie należy zakończyć:
```python
cursor.close()
cnx.close()
```

In [1]:
import mysql.connector
import random

### Nawiązanie połączenia z bazą 

In [2]:
db_name = 'cardb'

In [3]:
# słownik złożony z argumentów kluczowych funkcji connect()
config = {
  'user': 'root',
  'password': 'admin',
  'host': '127.0.0.1',
#   'database': db_name,
}

In [4]:
# połączenie się z serwerem (lub konkretną bazą na tym serwerze, jeżeli przekazany jest też argument database)
cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

In [5]:
# połączenie się z konkretną bazą
# jeżeli baza nie istnieje, zostaje utworzona
try:
    cnx.database = db_name
except mysql.connector.Error as err:
  if err.errno == mysql.connector.errorcode.ER_ACCESS_DENIED_ERROR:
    print('Something is wrong with your user name or password')
  elif err.errno == mysql.connector.errorcode.ER_BAD_DB_ERROR:
#     print('Database {} does not exist'.format(db_name))
    cursor.execute('CREATE DATABASE {}'.format(db_name))
    cnx.database = db_name
  else:
    print(err)

### Utworzenie tabel

In [6]:
table_dict = {}

In [7]:
table_dict['Car'] = '''
    CREATE TABLE `Car` (
    `Id` INT NOT NULL,
    `Model` VARCHAR(45) NOT NULL,
    PRIMARY KEY (`Id`))
    '''

table_dict['Part'] = '''
    CREATE TABLE `Part` (
    `Id` INT NOT NULL,
    `IsFunctional` TINYINT(1) NOT NULL,
    `CarId` INT,
    PRIMARY KEY (`Id`),
    FOREIGN KEY (`CarId`) REFERENCES `Car`(`Id`))
    '''

table_dict['Wheel'] = '''
    CREATE TABLE `Wheel` (
    `PartId` INT NOT NULL,
    `Model` VARCHAR(45) NOT NULL,
    PRIMARY KEY (`PartId`),
    FOREIGN KEY (`PartId`) REFERENCES `Part`(`Id`))
    '''

table_dict['Engine'] = '''
    CREATE TABLE `Engine` (
    `PartId` INT NOT NULL,
    `Model` VARCHAR(45) NOT NULL,
    PRIMARY KEY (`PartId`),
    FOREIGN KEY (`PartId`) REFERENCES `Part`(`Id`))
    '''

In [8]:
# utworzenie tabel
# jeżeli już istnieją, pojawi się komunikat
for table_name, create_table_sql_statement in table_dict.items():
    try:
        cursor.execute(create_table_sql_statement)
    except mysql.connector.Error as err:
      if err.errno == mysql.connector.errorcode.ER_TABLE_EXISTS_ERROR:
        print('Table {} already exists'.format(table_name))
      else:
        print(err)

### Dodanie wierszy

In [9]:
add_car_sql_statement = '''
    INSERT INTO Car (Id, Model)
    VALUES (%s, %s)
    '''

add_part_sql_statement = '''
    INSERT INTO Part (Id, IsFunctional, CarId)
    VALUES (%s, %s, %s)
    '''

add_wheel_sql_statement = '''
    INSERT INTO Wheel (PartId, Model)
    VALUES (%s, %s)
    '''

add_engine_sql_statement = '''
    INSERT INTO Engine (PartId, Model)
    VALUES (%s, %s)
    '''

In [10]:
car_total = 20
car_id_list = list(range(car_total))
car_model_list = ['Renault', 'Audi', 'BMW', 'Mazda', 'Fiat']
car_list = [(car_id, random.choice(car_model_list)) for car_id in car_id_list]

part_total = 100
part_id_list = list(range(part_total))
part_list = [(part_id, random.randint(0, 1), random.choice(car_id_list)) for part_id in part_id_list]

wheel_total = int(part_total * 0.7)
wheel_model_list = ['W1', 'W2', 'W3', 'W4', 'W5']
wheel_part_id_list = random.sample(part_id_list, wheel_total)
wheel_list = [(part_id, random.choice(wheel_model_list)) for part_id in wheel_part_id_list]

engine_total = part_total - wheel_total
engine_model_list = ['A157', 'B458', 'A86', 'A123', 'C18']
engine_part_id_list = list(set(part_id_list) - set(wheel_part_id_list))
engine_list = [(part_id, random.choice(engine_model_list)) for part_id in engine_part_id_list]

In [11]:
for car in car_list:
    cursor.execute(add_car_sql_statement, car)

In [12]:
for part in part_list:
    cursor.execute(add_part_sql_statement, part)

In [13]:
for wheel in wheel_list:
    cursor.execute(add_wheel_sql_statement, wheel)

In [14]:
for engine in engine_list:
    cursor.execute(add_engine_sql_statement, engine)

In [15]:
cnx.commit()

### Zaktualizowanie wierszy

In [16]:
set_free_part_sql_statement = '''
    UPDATE Part
    SET CarId = NULL
    WHERE Id IN {}
    '''

set_broken_part_sql_statement = '''
    UPDATE Part
    SET IsFunctional = 0
    WHERE Id = %s
    '''

set_fixed_part_sql_statement = '''
    UPDATE Part
    SET IsFunctional = 1
    WHERE Id = %s
    '''

In [17]:
free_part_id_list = random.sample(part_id_list, 15)
set_free_part_sql_statement = set_free_part_sql_statement.format(str(tuple(['%s' for i in range(len(free_part_id_list))])))

In [18]:
cursor.execute(set_free_part_sql_statement, free_part_id_list)

In [19]:
cnx.commit()

### Zapytania

In [20]:
# samochód jest sprawny, jeżeli wszystkie jego części są sprawne
get_functional_car_query = '''
    SELECT Car.Id, Model
    FROM Car INNER JOIN Part ON Car.Id = Part.CarId
    GROUP BY Car.Id
    HAVING COUNT(IsFunctional) = SUM(IsFunctional)
    '''

In [21]:
cursor.execute(get_functional_car_query)
for row in cursor:
    print(row)

(10, 'Fiat')
(13, 'Audi')
(18, 'Audi')


In [22]:
get_free_part_query = '''
    SELECT Part.Id AS Id, Model, 'Wheel' AS Type
    FROM Part INNER JOIN Wheel ON Part.Id = Wheel.PartId
    WHERE CarId IS NULL
    UNION
    SELECT Part.Id AS Id, Model, 'Engine' AS Type
    FROM Part INNER JOIN Engine ON Part.Id = Engine.PartId
    WHERE CarId IS NULL
    ORDER BY Id
    '''

In [23]:
cursor.execute(get_free_part_query)
for row in cursor:
    print(row)

(1, 'W2', 'Wheel')
(5, 'B458', 'Engine')
(12, 'C18', 'Engine')
(21, 'W2', 'Wheel')
(34, 'W3', 'Wheel')
(35, 'A157', 'Engine')
(36, 'W1', 'Wheel')
(47, 'W1', 'Wheel')
(54, 'W5', 'Wheel')
(60, 'W1', 'Wheel')
(61, 'B458', 'Engine')
(73, 'B458', 'Engine')
(90, 'W5', 'Wheel')
(92, 'W5', 'Wheel')
(98, 'A123', 'Engine')


### Zakończenie połączenia

In [24]:
cursor.close()
cnx.close()